# Gencode Annotation Parsing for RNASeq notebooks

* Art Nasamran, CCBB <cnasamran@ucsd.edu> 
* Adapted from notebook/code by: Kathleen Fisch, Ph.D. from "CCBB_RNAseq_Differential_Expression_Notebook.html/ipynb" circa 2017 and my gencodev28_annot.R script for Rattus_norvegicus.Rnor_6.0.92.gtf.
* March 2019
* **WARNING: Loading and parsing the gtf file is memory intensive. Not recommended for computers with <= 4GB of RAM.**

The gencode annotation file, more commonly seen as "ANNOT.Rdata" in CCBB's RNASeq notebooks, is primarily used as a reference to distinguish between coding and non-coding RNAs. This notebook parses a gencode annotation file and creates an Rdata reference. [Helpful table of gene/transcript biotypes](https://www.gencodegenes.org/pages/biotypes.html) for distinguishing protein coding vs non-coding genes.

# Load annotation file
* This notebook uses [Human annotation Release 29 (GRCh38.p12 Ensembl 94)](ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_29/gencode.v29.annotation.gtf.gz). However, any gencode annotation file should work with minimal code modifications.
* Downloaded the Comprehensive gene annotation; CHR; GTF file

In [2]:
## Do not run until location of gtf file specified ## 
gtf <- read.csv("../inputs/gencode.v29.annotation.gtf", skip = 5, header = FALSE, sep = "\t")
# First 5 lines contain description, provider, contact, format, and date information

In [3]:
head(gtf)

V1,V2,V3,V4,V5,V6,V7,V8,V9
<fct>,<fct>,<fct>,<int>,<int>,<fct>,<fct>,<fct>,<fct>
chr1,HAVANA,gene,11869,14409,.,+,.,gene_id ENSG00000223972.5; gene_type transcribed_unprocessed_pseudogene; gene_name DDX11L1; level 2; havana_gene OTTHUMG00000000961.2;
chr1,HAVANA,transcript,11869,14409,.,+,.,gene_id ENSG00000223972.5; transcript_id ENST00000456328.2; gene_type transcribed_unprocessed_pseudogene; gene_name DDX11L1; transcript_type processed_transcript; transcript_name DDX11L1-202; level 2; transcript_support_level 1; tag basic; havana_gene OTTHUMG00000000961.2; havana_transcript OTTHUMT00000362751.1;
chr1,HAVANA,exon,11869,12227,.,+,.,gene_id ENSG00000223972.5; transcript_id ENST00000456328.2; gene_type transcribed_unprocessed_pseudogene; gene_name DDX11L1; transcript_type processed_transcript; transcript_name DDX11L1-202; exon_number 1; exon_id ENSE00002234944.1; level 2; transcript_support_level 1; tag basic; havana_gene OTTHUMG00000000961.2; havana_transcript OTTHUMT00000362751.1;
chr1,HAVANA,exon,12613,12721,.,+,.,gene_id ENSG00000223972.5; transcript_id ENST00000456328.2; gene_type transcribed_unprocessed_pseudogene; gene_name DDX11L1; transcript_type processed_transcript; transcript_name DDX11L1-202; exon_number 2; exon_id ENSE00003582793.1; level 2; transcript_support_level 1; tag basic; havana_gene OTTHUMG00000000961.2; havana_transcript OTTHUMT00000362751.1;
chr1,HAVANA,exon,13221,14409,.,+,.,gene_id ENSG00000223972.5; transcript_id ENST00000456328.2; gene_type transcribed_unprocessed_pseudogene; gene_name DDX11L1; transcript_type processed_transcript; transcript_name DDX11L1-202; exon_number 3; exon_id ENSE00002312635.1; level 2; transcript_support_level 1; tag basic; havana_gene OTTHUMG00000000961.2; havana_transcript OTTHUMT00000362751.1;
chr1,HAVANA,transcript,12010,13670,.,+,.,gene_id ENSG00000223972.5; transcript_id ENST00000450305.2; gene_type transcribed_unprocessed_pseudogene; gene_name DDX11L1; transcript_type transcribed_unprocessed_pseudogene; transcript_name DDX11L1-201; level 2; transcript_support_level NA; ont PGO:0000005; ont PGO:0000019; tag basic; havana_gene OTTHUMG00000000961.2; havana_transcript OTTHUMT00000002844.2;


In [4]:
table(gtf$V2)


ENSEMBL  HAVANA 
 251831 2490186 

In [8]:
# Add column names
names(gtf) <- c("chr", "database", "gene_location", "start", "stop", "blank2", "strand", "blank3", "info")

In [18]:
# Parse info column by ;
require(splitstackshape)
x <- cSplit(gtf, "info", sep = ";")

In [19]:
head(x)

chr,database,gene_location,start,stop,blank2,strand,blank3,info_01,info_02,⋯,info_13,info_14,info_15,info_16,info_17,info_18,info_19,info_20,info_21,info_22
<fct>,<fct>,<fct>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
chr1,HAVANA,gene,11869,14409,.,+,.,gene_id ENSG00000223972.5,gene_type transcribed_unprocessed_pseudogene,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
chr1,HAVANA,transcript,11869,14409,.,+,.,gene_id ENSG00000223972.5,transcript_id ENST00000456328.2,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
chr1,HAVANA,exon,11869,12227,.,+,.,gene_id ENSG00000223972.5,transcript_id ENST00000456328.2,⋯,havana_transcript OTTHUMT00000362751.1,NA,NA,NA,NA,NA,NA,NA,NA,NA
chr1,HAVANA,exon,12613,12721,.,+,.,gene_id ENSG00000223972.5,transcript_id ENST00000456328.2,⋯,havana_transcript OTTHUMT00000362751.1,NA,NA,NA,NA,NA,NA,NA,NA,NA
chr1,HAVANA,exon,13221,14409,.,+,.,gene_id ENSG00000223972.5,transcript_id ENST00000456328.2,⋯,havana_transcript OTTHUMT00000362751.1,NA,NA,NA,NA,NA,NA,NA,NA,NA
chr1,HAVANA,transcript,12010,13670,.,+,.,gene_id ENSG00000223972.5,transcript_id ENST00000450305.2,⋯,havana_transcript OTTHUMT00000002844.2,NA,NA,NA,NA,NA,NA,NA,NA,NA


## Filter annotations by transcript
We're interested in retaining the transcript annotations because they contain more detail than the gene annotations. Furthermore, the transcript annotations are useful for isoform-level analyses.

In [7]:
table(gtf$gene_location)


           CDS           exon           gene Selenocysteine    start_codon 
        746753        1262773          58721            120          86445 
    stop_codon     transcript            UTR 
         78574         206694         301937 

In [20]:
ex <- x[which(x$gene_location=="exon"),]
dim(ex)

[1] 1262773      30

In [21]:
x <- x[which(x$gene_location == "transcript"),]
dim(x)

[1] 206694     30

In [9]:
head(x)[,1:14]
head(x)[,15:30]
dim(x)
colnames(x)

chr,database,gene_location,start,stop,blank2,strand,blank3,info_01,info_02,info_03,info_04,info_05,info_06
chr1,HAVANA,transcript,11869,14409,.,+,.,gene_id ENSG00000223972.5,transcript_id ENST00000456328.2,gene_type transcribed_unprocessed_pseudogene,gene_name DDX11L1,transcript_type processed_transcript,transcript_name DDX11L1-202
chr1,HAVANA,transcript,12010,13670,.,+,.,gene_id ENSG00000223972.5,transcript_id ENST00000450305.2,gene_type transcribed_unprocessed_pseudogene,gene_name DDX11L1,transcript_type transcribed_unprocessed_pseudogene,transcript_name DDX11L1-201
chr1,HAVANA,transcript,14404,29570,.,-,.,gene_id ENSG00000227232.5,transcript_id ENST00000488147.1,gene_type unprocessed_pseudogene,gene_name WASH7P,transcript_type unprocessed_pseudogene,transcript_name WASH7P-201
chr1,ENSEMBL,transcript,17369,17436,.,-,.,gene_id ENSG00000278267.1,transcript_id ENST00000619216.1,gene_type miRNA,gene_name MIR6859-1,transcript_type miRNA,transcript_name MIR6859-1-201
chr1,HAVANA,transcript,29554,31097,.,+,.,gene_id ENSG00000243485.5,transcript_id ENST00000473358.1,gene_type lincRNA,gene_name MIR1302-2HG,transcript_type lincRNA,transcript_name MIR1302-2HG-202
chr1,HAVANA,transcript,30267,31109,.,+,.,gene_id ENSG00000243485.5,transcript_id ENST00000469289.1,gene_type lincRNA,gene_name MIR1302-2HG,transcript_type lincRNA,transcript_name MIR1302-2HG-201


info_07,info_08,info_09,info_10,info_11,info_12,info_13,info_14,info_15,info_16,info_17,info_18,info_19,info_20,info_21,info_22
level 2,transcript_support_level 1,tag basic,havana_gene OTTHUMG00000000961.2,havana_transcript OTTHUMT00000362751.1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
level 2,transcript_support_level NA,ont PGO:0000005,ont PGO:0000019,tag basic,havana_gene OTTHUMG00000000961.2,havana_transcript OTTHUMT00000002844.2,NA,NA,NA,NA,NA,NA,NA,NA,NA
level 2,transcript_support_level NA,ont PGO:0000005,tag basic,havana_gene OTTHUMG00000000958.1,havana_transcript OTTHUMT00000002839.1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
level 3,transcript_support_level NA,tag basic,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
level 2,transcript_support_level 5,tag not_best_in_genome_evidence,tag dotter_confirmed,tag basic,havana_gene OTTHUMG00000000959.2,havana_transcript OTTHUMT00000002840.1,NA,NA,NA,NA,NA,NA,NA,NA,NA
level 2,transcript_support_level 5,tag not_best_in_genome_evidence,tag basic,havana_gene OTTHUMG00000000959.2,havana_transcript OTTHUMT00000002841.2,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


[1] 206694     30

[1] "chr"           "database"      "gene_location" "start"        
 [5] "stop"          "blank2"        "strand"        "blank3"       
 [9] "info_01"       "info_02"       "info_03"       "info_04"      
[13] "info_05"       "info_06"       "info_07"       "info_08"      
[17] "info_09"       "info_10"       "info_11"       "info_12"      
[21] "info_13"       "info_14"       "info_15"       "info_16"      
[25] "info_17"       "info_18"       "info_19"       "info_20"      
[29] "info_21"       "info_22"

# Parse x object

## Remove descriptive text from data
* Look at the headers of the previous cell to find the appropriate names for columns of interest.

In [22]:
x$info_01 <- sub("gene_id ", "", x$info_01)
x$info_02 <- sub("transcript_id ", "", x$info_02)
x$info_03 <- sub("gene_type ", "", x$info_03)
#x$info_04 <- sub("gene_name ", "", x$info_04)
#x$info_05 <- sub("transcript_type ", "", x$info_05)
#x$info_06 <- sub("transcript_name ", "", x$info_06)
#x$info_07 <- sub("exon_number ", "", x$info_07)
#x$info_08 <- sub("exon_id ", "", x$info_08)

ex$info_01 <- sub("gene_id ", "", ex$info_01)
ex$info_02 <- sub("transcript_id ", "", ex$info_02)
ex$info_03 <- sub("gene_type ", "", ex$info_03)
ex$info_04 <- sub("gene_name ", "", ex$info_04)
ex$info_05 <- sub("transcript_type ", "", ex$info_05)
ex$info_06 <- sub("transcript_name ", "", ex$info_06)
ex$info_07 <- sub("exon_number ", "", ex$info_07)
ex$info_08 <- sub("exon_id ", "", ex$info_08)


# Create ANNOT object

In [23]:
ANNOT <- data.frame("gene_type" = x$info_03,
                    "gene_id" = x$info_01,
                    "transcript_id" = x$info_02)

ANNOT_ex <- data.frame("chr" = ex$chr, 
                       "start" = ex$start,
                       "stop" = ex$stop,
                       "stand" = ex$strand,
                       "gene_type" = ex$info_03,
                       "gene_id" = ex$info_01,
                       "transcript_id" = ex$info_02, 
                      "gene_name" = ex$info_04, 
                      "transcript_type" = ex$info_05, 
                      "exon_number" = ex$info_07,
                      "exon_id" = ex$info_08)

## Remove white space from character columns

In [24]:
#ANNOT$type <- gsub(" ", "", ANNOT$type)
#ANNOT$gene_location <- gsub(" ", "", ANNOT$gene_location)
ANNOT$gene_id <- gsub(" ", "", ANNOT$gene_id)
ANNOT$transcript_id <- gsub(" ", "", ANNOT$transcript_id)
ANNOT$gene_type <- gsub(" ", "", ANNOT$gene_type)
#ANNOT$gene_name <- gsub(" ", "", ANNOT$gene_name)

#ANNOT_ex$type <- gsub(" ", "", ANNOT_ex$type)
#ANNOT_ex$gene_location <- gsub(" ", "", ANNOT_ex$gene_location)
ANNOT_ex$gene_id <- gsub(" ", "", ANNOT_ex$gene_id)
ANNOT_ex$transcript_id <- gsub(" ", "", ANNOT_ex$transcript_id)
ANNOT_ex$gene_type <- gsub(" ", "", ANNOT_ex$gene_type)
ANNOT_ex$gene_name <- gsub(" ", "", ANNOT_ex$gene_name)
ANNOT_ex$transcript_type <- gsub(" ", "", ANNOT_ex$transcript_type)
ANNOT_ex$exon_number <- gsub(" ", "", ANNOT_ex$exon_number)
ANNOT_ex$exon_id <- gsub(" ", "", ANNOT_ex$exon_id)

In [16]:
table(ANNOT$gene_type)


          3prime_overlapping_ncRNA                          antisense 
                                38                              10330 
     bidirectional_promoter_lncRNA                          IG_C_gene 
                               273                                 24 
                   IG_C_pseudogene                          IG_D_gene 
                                 9                                 37 
                         IG_J_gene                    IG_J_pseudogene 
                                18                                  3 
                     IG_pseudogene                          IG_V_gene 
                                 1                                153 
                   IG_V_pseudogene                            lincRNA 
                               188                              13188 
                      macro_lncRNA                              miRNA 
                                 1                               1881 
     

In [27]:
table(ANNOT_ex$gene_type)
dim(ANNOT_ex)
length(unique(ANNOT_ex$gene_id))


          3prime_overlapping_ncRNA                          antisense 
                                78                              30719 
     bidirectional_promoter_lncRNA                          IG_C_gene 
                              1122                                105 
                   IG_C_pseudogene                          IG_D_gene 
                                15                                 37 
                         IG_J_gene                    IG_J_pseudogene 
                                18                                  3 
                     IG_pseudogene                          IG_V_gene 
                                 1                                301 
                   IG_V_pseudogene                            lincRNA 
                               288                              41163 
                      macro_lncRNA                              miRNA 
                                 1                               1881 
     

[1] 1262773      11

[1] 58721

In [26]:
length(unique(ANNOT$gene_id))

[1] 58721

## Remove pseudoautosomal regions (PAR)
* [PAR](https://uswest.ensembl.org/info/genome/genebuild/human_PARS.html) occur where chromosome X and Y share homologous sequences. These genes are not relevant to the protein coding and non-protein coding designations that we use the annotation file for and can be removed. They also disrupt the RNASeq notebook from properly parsing coding and non-coding gene lists.
* PAR are designated with a _PAR_X/Y suffix

In [29]:
par.idx <- grep("*_PAR_*", ANNOT$gene_id)
par.idx.ex <- grep("*_PAR_*", ANNOT_ex$gene_id)

In [30]:
head(ANNOT[par.idx,], 5)
length(unique(ANNOT[par.idx, "gene_id"]))
head(ANNOT_ex[par.idx.ex,], 5)
length(unique(ANNOT_ex[par.idx.ex, "gene_id"]))

,gene_type,gene_id,transcript_id
,<chr>,<chr>,<chr>
205760,unprocessed_pseudogene,ENSG00000228572.7_PAR_Y,ENST00000431238.7_PAR_Y
205761,protein_coding,ENSG00000182378.13_PAR_Y,ENST00000399012.6_PAR_Y
205762,protein_coding,ENSG00000182378.13_PAR_Y,ENST00000484611.7_PAR_Y
205763,protein_coding,ENSG00000182378.13_PAR_Y,ENST00000430923.7_PAR_Y
205764,protein_coding,ENSG00000182378.13_PAR_Y,ENST00000445062.6_PAR_Y


[1] 45

,chr,start,stop,stand,gene_type,gene_id,transcript_id,gene_name,transcript_type,exon_number,exon_id
,<fct>,<int>,<int>,<fct>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1258107,chrY,253743,253846,+,unprocessed_pseudogene,ENSG00000228572.7_PAR_Y,ENST00000431238.7_PAR_Y,AL954722.1,unprocessed_pseudogene,1,ENSE00001702291.1
1258108,chrY,254937,255091,+,unprocessed_pseudogene,ENSG00000228572.7_PAR_Y,ENST00000431238.7_PAR_Y,AL954722.1,unprocessed_pseudogene,2,ENSE00001655436.1
1258109,chrY,276322,276394,+,protein_coding,ENSG00000182378.13_PAR_Y,ENST00000399012.6_PAR_Y,PLCXD1,protein_coding,1,ENSE00001489430.1
1258110,chrY,281482,281684,+,protein_coding,ENSG00000182378.13_PAR_Y,ENST00000399012.6_PAR_Y,PLCXD1,protein_coding,2,ENSE00001306908.2
1258111,chrY,284167,284314,+,protein_coding,ENSG00000182378.13_PAR_Y,ENST00000399012.6_PAR_Y,PLCXD1,protein_coding,3,ENSE00003812308.1


[1] 45

In [31]:
dim(ANNOT)
ANNOT <- ANNOT[-par.idx,]
dim(ANNOT)

dim(ANNOT_ex)
ANNOT_ex <- ANNOT_ex[-par.idx.ex,]
dim(ANNOT_ex)

[1] 206694      3

[1] 206534      3

[1] 1262773      11

[1] 1261907      11

## Remove duplicate rows

In [24]:
head(ANNOT,10)
dim(ANNOT)

gene_type,gene_id,transcript_id
transcribed_unprocessed_pseudogene,ENSG00000223972.5,ENST00000456328.2
transcribed_unprocessed_pseudogene,ENSG00000223972.5,ENST00000450305.2
unprocessed_pseudogene,ENSG00000227232.5,ENST00000488147.1
miRNA,ENSG00000278267.1,ENST00000619216.1
lincRNA,ENSG00000243485.5,ENST00000473358.1
lincRNA,ENSG00000243485.5,ENST00000469289.1
miRNA,ENSG00000284332.1,ENST00000607096.1
lincRNA,ENSG00000237613.2,ENST00000417324.1
lincRNA,ENSG00000237613.2,ENST00000461467.1
unprocessed_pseudogene,ENSG00000268020.3,ENST00000606857.1


[1] 206534      3

In [25]:
ANNOT <- unique(ANNOT[,1:3])

In [26]:
head(ANNOT,10)
dim(ANNOT)

gene_type,gene_id,transcript_id
transcribed_unprocessed_pseudogene,ENSG00000223972.5,ENST00000456328.2
transcribed_unprocessed_pseudogene,ENSG00000223972.5,ENST00000450305.2
unprocessed_pseudogene,ENSG00000227232.5,ENST00000488147.1
miRNA,ENSG00000278267.1,ENST00000619216.1
lincRNA,ENSG00000243485.5,ENST00000473358.1
lincRNA,ENSG00000243485.5,ENST00000469289.1
miRNA,ENSG00000284332.1,ENST00000607096.1
lincRNA,ENSG00000237613.2,ENST00000417324.1
lincRNA,ENSG00000237613.2,ENST00000461467.1
unprocessed_pseudogene,ENSG00000268020.3,ENST00000606857.1


[1] 206534      3

In [27]:
table(ANNOT$gene_type)


          3prime_overlapping_ncRNA                          antisense 
                                38                              10322 
     bidirectional_promoter_lncRNA                          IG_C_gene 
                               273                                 24 
                   IG_C_pseudogene                          IG_D_gene 
                                 9                                 37 
                         IG_J_gene                    IG_J_pseudogene 
                                18                                  3 
                     IG_pseudogene                          IG_V_gene 
                                 1                                153 
                   IG_V_pseudogene                            lincRNA 
                               188                              13180 
                      macro_lncRNA                              miRNA 
                                 1                               1879 
     

# Save ANNOT file in .Rdata format for fast loading
* **Please include species name, reference genome name and version, and the gencode version in the file name.**
* Also store the source URL for your gtf file

In [34]:
gtfSourceURL <- "ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_29/gencode.v29.annotation.gtf.gz"

In [29]:
save(ANNOT, gtfSourceURL, file = "Homo_sapiens_GRCh38p12_gencodev29_ANNOT.Rdata")

In [35]:
save(ANNOT_ex, file = "../inputs/Homo_sapiens_GRCh38p12_gencodev29_exons_ANNOT.Rdata")


## Software and code used

In [30]:
sessionInfo()

R version 3.4.3 (2017-11-30)
Platform: x86_64-redhat-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /usr/lib64/R/lib/libRblas.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] splitstackshape_1.4.6

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.0           digest_0.6.17        crayon_1.3.4        
 [4] IRdisplay_0.5.0      repr_0.15.0          jsonlite_1.6        
 [7] magrittr_1.5         evaluate_0.11        stringi_1.2.4       
[10] uuid_0.1-2           data.table_1.12.0    IRkernel_0.8.1

2019 UC San Diego Center for Computational Biology & Bioinformatics

Notebook by Art Nasamran <cnasamran@ucsd.edu>